In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.externals import joblib
from sklearn.svm import SVC

In [2]:
data = pd.read_csv("amannew.csv")

In [3]:
print data.head()

   docID  authorID   paraID       f1        f2        f3       f4   f5  f6  \
0   7042      2786  8767356  49.4659  0.112412 -0.139156  628.772  854   0   
1   7042      2786  8767357  50.9139  0.100802 -0.139333  632.473  873   0   
2   7042      2786  8767358  48.6102  0.080447 -0.139319  636.327  895   0   
3   7042      2786  8767359  42.4379  0.089623 -0.139467  632.219  848   0   
4   7042      2786  8767360  47.6189  0.086801 -0.139201  627.527  841   0   

    f7  f8  f9  f10  
0  156   2   0   15  
1  170   3   0   13  
2  171   4   0   12  
3  185   8   0   19  
4  167   7   0   20  


In [4]:
print data.shape 

(391572, 13)


In [5]:
print(data.dtypes)

docID         int64
authorID      int64
paraID        int64
f1          float64
f2          float64
f3          float64
f4          float64
f5            int64
f6            int64
f7            int64
f8            int64
f9            int64
f10           int64
dtype: object


In [6]:
def getData(data):
    labels = []
    groups = []
    features = []
    size = []
    authorList = data.authorID.unique()
    for auth in authorList:
        current = data.loc[data['authorID'] == auth]
        size.append(current.shape[0])

    print("Min: %s" % (min(size)))
    print("Max: %s" % (max(size)))

    authorList = authorList.tolist()

    for auth in authorList:
        current = data.loc[data['authorID'] == auth]

        # current = current.sample(n = samples)
        feat = current[["f1", "f2", "f3", "f4", 
                        "f5", "f6", "f7", "f8",
                        "f9", "f10"]].values.tolist()
        features = features + feat
        author = current["authorID"].tolist()
        labels = labels + author
        doc = current["docID"].tolist()
        groups = groups + doc
        

    return (np.array(features), np.array(labels), np.array(groups))


In [7]:
tests = pd.read_csv("finalA6.csv")
print(tests.dtypes)
authorList = tests.author_id.unique().tolist()

print type(authorList)


author_id     int64
doc_id        int64
lang         object
count         int64
dtype: object
<type 'list'>


In [8]:
df = pd.DataFrame()

for index, row in tests.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id
    
    train = data.loc[data['authorID'] == author_id]
    
    df = pd.concat([df, train])
    
print df.shape

data = df.copy()

del df

print data.authorID.unique()

(17620, 13)
[ 114 1492  124 1228 1472]


In [9]:

def predictModel(model, testX):
    # Function to take input of data and return prediction model
    predY = np.array(model.predict_proba(testX))

    predYList = predY[:]
    
    print len(predYList)
    entro = []

    flag = False
    import math
    for row in predY:
        entroval = 0
        for i in row:
            if(i <= 0):
                flag = True
                pass
            else:
                entroval += (i * (math.log(i , 2)))
        entroval = -1 * entroval
        entro.append(entroval)

    if(flag == False):
        yx = zip(entro, predY)
        yx = sorted(yx, key = lambda t: t[0])
        newPredY = [x for y, x in yx]
        predYEntroList = newPredY[:int(len(newPredY)*0.3)] # Reduce this 
        predY = np.mean(predYEntroList, axis=0)
    else:
        predY = np.mean(predYList, axis=0)

    return (predYList, predY)

results = []
test_binary = []
for index, row in tests.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    train = data.loc[data['docID'] != doc_id]

    test = data.loc[data['docID'] == doc_id]

    (train_features, train_labels, train_groups) = getData(train)

    (test_features, test_labels, test_groups) = getData(test)

    del train

    ## SVC
    clf = SVC(kernel="linear", probability=True, verbose=True)
    clf.fit(train_features, train_labels)

    (predYList, predY) = predictModel(clf, test_features)
    testY = np.array(predY)
    testY = testY.mean(axis = 0)
    predLocation = predY.tolist().index(max(predY))
    if predLocation == testY:
        test_binary.append(1)
    else:
        test_binary.append(0)
        
    del clf
    
    from IPython.display import clear_output
    clear_output()
    
print("Complete")


Complete


In [10]:
test_binary

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [11]:
results = []
for index, row in tests.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id
    
    train = data.loc[data['docID'] != doc_id]
    
    test = data.loc[data['docID'] == doc_id]
    
    (train_features, train_labels, train_groups) = getData(train)
    
    (test_features, test_labels, test_groups) = getData(test)
    
    del train
    
    ## SVC
    clf = SVC(kernel='linear', verbose= True, C= 1)
    clf.fit(train_features, train_labels)
    
    print 'Train Accuracy'
    print clf.score(train_features, train_labels)

    print 'Test Accuracy'
    ta = clf.score(test_features, test_labels)
    print ta
    results.append(ta)

    

    

Min: 944
Max: 9000
Min: 460
Max: 460
[LibSVM]Train Accuracy
0.827505827506
Test Accuracy
0.402173913043
Min: 944
Max: 9000
Min: 1430
Max: 1430
[LibSVM]Train Accuracy
0.858801729463
Test Accuracy
0.146853146853
Min: 944
Max: 9000
Min: 366
Max: 366
[LibSVM]Train Accuracy
0.826822765735
Test Accuracy
0.0
Min: 366
Max: 9000
Min: 1014
Max: 1014
[LibSVM]Train Accuracy
0.833313260267
Test Accuracy
0.0
Min: 496
Max: 9000
Min: 448
Max: 448
[LibSVM]Train Accuracy
0.843349638947
Test Accuracy
0.0
Min: 448
Max: 9000
Min: 496
Max: 496
[LibSVM]Train Accuracy
0.822237794908
Test Accuracy
0.0
Min: 576
Max: 9000
Min: 570
Max: 570
[LibSVM]Train Accuracy
0.836598240469
Test Accuracy
0.0
Min: 570
Max: 9000
Min: 576
Max: 576
[LibSVM]Train Accuracy
0.832433701009
Test Accuracy
0.0
Min: 944
Max: 9000
Min: 460
Max: 460
[LibSVM]Train Accuracy
0.827505827506
Test Accuracy
0.402173913043
Min: 944
Max: 8196
Min: 804
Max: 804
[LibSVM]Train Accuracy
0.828853472883
Test Accuracy
0.417910447761
Min: 944
Max: 9000
Min

In [12]:
results

[0.40217391304347827,
 0.14685314685314685,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40217391304347827,
 0.41791044776119401,
 0.14685314685314685,
 0.0,
 0.69696969696969702,
 0.0,
 0.87855297157622736]

In [13]:
np.average(results)

0.20609914907335794